In [57]:
# import package
import os
import pandas as pd
import numpy as np
from scipy.sparse import lil_matrix
import math
import sys
import random

# mount colab on google drive
from google.colab import drive
drive.mount('/content/drive')

# change dir, root dir
os.chdir('/content')

# change dir, file dir
os.chdir('drive/My Drive/codes/socomp')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [58]:
# read GroupTopic.txt
gt_lines = []
with open('GroupTopic.txt') as gt_f:
    gt_lines = gt_f.readlines()
gt_f.close()

group_id = []
group_organizer = []
group_attribute = []
gt_count = 1
for gt_line in gt_lines:
    if gt_count % 2 == 1:
      gt_id, gt_organizer = gt_line.split()
      group_id.append(int(gt_id[1:]))
      if gt_organizer == 'null':
        group_organizer.append(int(-1))
      else:
        group_organizer.append(int(gt_organizer[1:]))
    else:
      read_one_group_attribute = gt_line.split()
      num_group_attribute_list = []
      for i in range(len(read_one_group_attribute)):
        
        num_group_attribute_list.append(
            int(read_one_group_attribute[i][1:]))
        
      group_attribute.append(num_group_attribute_list)
    gt_count += 1

gt_data = zip(group_id, group_organizer, group_attribute)

gt_df = pd.DataFrame(
    gt_data, columns = ['group_id', 'group_organizer', 'group_attribute'])

gt_df = gt_df[gt_df['group_organizer'] != -1]
max_group_id = gt_df['group_id'].max()
group_amount = len(gt_df.index)
gt_df

,group_id,group_organizer,group_attribute
0,0,48462,"[85, 89, 124, 234, 738, 2527, 3007, 3135, 5915..."
1,1,41976,"[212, 512, 3554]"
2,2,51926,[1566]
3,3,48348,"[244, 309, 1322, 1502, 3336, 4422, 8649, 9816,..."
4,4,734,"[638, 9753, 9772, 15067, 15984, 18874, 19061]"
...,...,...,...
477,477,21201,"[1429, 7929, 18340, 29702]"
478,478,50243,"[484, 1307, 2064, 3880, 6364, 10117, 15236, 15..."
479,479,35028,"[9772, 10002, 10024, 10413]"
480,480,30753,"[1237, 1273, 1543, 10050, 10080, 10232, 17665,..."


In [59]:
# read MemberTopic.txt
mt_lines = []
with open('MemberTopic.txt') as mt_f:
    mt_lines = mt_f.readlines()
mt_f.close()

member_id = []
member_attribute = []
mt_count = 1
for mt_line in mt_lines:
    if mt_count % 2 == 1:
      member_id.append(int(mt_line[1:-1]))
    else:
      read_one_member_attribute = mt_line.split()
      num_member_attribute_list = []
      for i in range(len(read_one_member_attribute)):
        
        num_member_attribute_list.append(
            int(read_one_member_attribute[i][1:]))
        
      member_attribute.append(num_member_attribute_list)
    mt_count += 1

mt_data = zip(member_id, member_attribute)

mt_df = pd.DataFrame(
    mt_data, columns = ['member_id', 'member_attribute'])

mt_df = mt_df[mt_df.astype(str)['member_attribute'] != '[]']
max_member_id = mt_df['member_id'].max()
member_amount = len(mt_df.index)
mt_df

,member_id,member_attribute
0,0,"[10485, 498]"
1,1,"[1201, 914, 933, 638, 590, 726, 242, 1724, 158..."
2,2,"[10495, 9753, 491, 1201, 638, 2926, 15892, 229..."
3,3,"[11660, 11046, 2151]"
4,4,"[1201, 933, 230, 2899, 1755, 3282, 1923, 228, ..."
...,...,...
73677,73677,[10050]
73680,73680,"[1155, 17195, 60, 17866, 10232, 17030, 713, 24..."
73681,73681,"[240, 3336, 495, 22440, 29431, 16551, 4422, 24..."
73682,73682,"[6761, 6490, 1526, 1307, 1330, 2088, 2064, 445..."


In [60]:
# read Group-Event folder
group_event_id = []
event_id = []
event_limit = []
event_time = []
event_organizer = []
yes_id = []
no_id = []
maybe_id = []
root = "./GroupEvent"
for i in range(0, 482):
  file = "G" + str(i) + ".txt"
  full_path = os.path.join(root, file)
  ge_group_id = file[:-4]
  with open(full_path) as ge_f:
    ge_lines = ge_f.readlines()
  ge_f.close()
  ge_count = 0
  for ge_line in ge_lines:
    ge_count += 1
    if ge_count % 5 == 0:
      maybe_id.append(ge_line.split())
    elif ge_count % 4 == 0:
      read_one_no_id = ge_line.split()
      num_no_id_list = []
      for i in range(len(read_one_no_id)):
        if read_one_no_id[i] != 'null':
          num_no_id_list.append(int(read_one_no_id[i][1:]))
      no_id.append(num_no_id_list)
    elif ge_count % 3 == 0:
      read_one_yes_id = ge_line.split()
      num_yes_id_list = []
      for i in range(len(read_one_yes_id)):
        if read_one_yes_id[i] != 'null':
          num_yes_id_list.append(int(read_one_yes_id[i][1:]))
      yes_id.append(num_yes_id_list)
    elif ge_count % 2 == 0:
      event_organizer.append(ge_line.split())
    else:
      ge_event_id, ge_event_limit, ge_event_time = ge_line.split()
      event_id.append(int(ge_event_id[1:]))
      event_limit.append(ge_event_limit)
      event_time.append(ge_event_time)
      group_event_id.append(int(ge_group_id[1:]))
    if ge_count == 5:
      ge_count = 0

ge_data = zip(
    group_event_id, event_id, event_limit, event_time, 
    event_organizer, yes_id, no_id, maybe_id)

ge_df = pd.DataFrame(
    ge_data, columns = [
      'group_id', 'event_id', 'event_limit', 'event_time', 
      'event_organizer', 'yes_id', 'no_id', 'maybe_id'])

ge_df[['event_limit', 'event_time']] = ge_df[
  ['event_limit', 'event_time']].astype(int)

# remove column maybe_id
ge_df = ge_df.drop('maybe_id', 1)

# remove column event_organizer
ge_df = ge_df.drop('event_organizer', 1)

# remove rows that yes_id==null && no_id==null
ge_df = ge_df[
  (ge_df.astype(str)['yes_id'] != '[]') & 
  (ge_df.astype(str)['no_id'] != '[]')]

max_event_id = ge_df['event_id'].max()
event_amount = len(ge_df.index) 
ge_df

,group_id,event_id,event_limit,event_time,yes_id,no_id
0,0,932,4,1240153200000,[48462],[65542]
1,0,947,6,1359820800000,"[48462, 56494, 62049, 70873, 48571]","[46348, 49649]"
2,0,1742,2,1244991600000,[48462],"[58371, 65542, 68509, 70851]"
4,0,3373,0,1363030200000,"[48462, 21224, 27288, 63893]",[51822]
5,0,6675,3,1255179600000,"[31584, 48462]",[65542]
...,...,...,...,...,...,...
55391,481,39040,26,1228435200000,"[24063, 30598, 53015, 54522, 55322, 55840, 565...","[33527, 53621, 63290, 64179]"
55392,481,40188,12,1232668800000,"[24063, 30598, 34608, 41229, 45326, 46797, 553...","[34367, 62320, 65316, 66040, 66317]"
55393,481,40672,7,1232668800000,"[30598, 38405, 45326, 52699, 55322, 56568, 622...","[33527, 53621, 59324, 64777]"
55394,481,41507,11,1235692800000,"[30598, 55322, 55850]","[23171, 24596, 43301, 45326, 46134, 52699, 593..."


In [61]:
# randomly shuffle the rows of ge_df
ge_df = ge_df.sample(frac=1)

# split ge_df into 5 parts
p1, p2, p3, p4, p5 = np.array_split(ge_df, 5)

# train_set_1, test_set_1
train_frames_1 = [p2, p3, p4, p5]
train_set_1 = pd.concat(train_frames_1)
test_set_1 = p1

train_frames_2 = [p1, p3, p4, p5]
train_set_2 = pd.concat(train_frames_2)
test_set_2 = p2

train_frames_3 = [p1, p2, p4, p5]
train_set_3 = pd.concat(train_frames_3)
test_set_3 = p3

train_frames_4 = [p1, p2, p3, p5]
train_set_4 = pd.concat(train_frames_4)
test_set_4 = p4

train_frames_5 = [p1, p2, p3, p4]
train_set_5 = pd.concat(train_frames_5)
test_set_5 = p5

train_set_1

,group_id,event_id,event_limit,event_time,yes_id,no_id
28173,277,980,2,1241308800000,[36083],"[30531, 34384]"
704,10,35099,2,1215558000000,"[39523, 41072]",[41506]
28876,278,46784,0,1334404800000,"[41552, 59685, 61936, 22176, 23178, 28050, 326...","[10815, 13154, 24953]"
52997,453,41526,21,1356739200000,"[13620, 23035, 32432, 9015, 14350, 45403, 51802]","[42269, 65667, 4578, 14282]"
32655,299,7236,8,1368568800000,"[60470, 2055]","[39920, 69165]"
...,...,...,...,...,...,...
10058,151,36354,0,1349303400000,"[7168, 36413, 4606, 8990, 12374, 23841, 28695,...","[62481, 70798, 317, 16902, 33260, 35879, 47029..."
21830,234,12053,6,1281738600000,"[63308, 30835, 67934]","[61277, 72088, 3446, 10891]"
36132,317,25654,30,1332633600000,"[40940, 57242, 60251, 4603, 9524, 11389, 15112...","[59518, 2722, 11492, 11909, 26510]"
42752,358,9364,2,1269108000000,"[27296, 27641]","[30355, 70401]"


In [62]:
# matrix member-event, UE_train
UE_train = lil_matrix((max_member_id+1, max_event_id+1), dtype=np.int8)
for i in range(len(train_set_1)):
  UE_train_c = train_set_1.iloc[i]['event_id']
  UE_train_r_set = train_set_1.iloc[i]['yes_id']
  for j in range(len(UE_train_r_set)):
    UE_train_r = UE_train_r_set[j]
    UE_train[UE_train_r, UE_train_c] = 1
UE_train.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int8)

In [63]:
# group_attribute_set
group_attribute_set = []
for i in range(group_amount):
  one_group_attribute = gt_df.iloc[i]['group_attribute']
  for j in range(len(one_group_attribute)):
    if one_group_attribute[j] not in group_attribute_set:
      group_attribute_set.append(one_group_attribute[j])
print(len(group_attribute_set))

1132


In [64]:
# member_attribute_set
member_attribute_set = []
for i in range(member_amount):
  one_member_attribute = mt_df.iloc[i]['member_attribute']
  for j in range(len(one_member_attribute)):
    if one_member_attribute[j] not in member_attribute_set:
      member_attribute_set.append(one_member_attribute[j])
print(len(member_attribute_set))

17106


In [65]:
# member and group common attribute
gm_common_attribute = list(
    set(group_attribute_set).intersection(member_attribute_set))
print(len(gm_common_attribute))

1110


In [66]:
# remove member_attribute (not in gm_common_attribute)
new_member_id = []
new_member_attribute = []
for i in range(member_amount):
  new_member_id.append(mt_df.iloc[i]['member_id'])
  old_member_attribute = mt_df.iloc[i]['member_attribute']
  one_new_member_attribute = list(
    set(gm_common_attribute).intersection(old_member_attribute))
  new_member_attribute.append(one_new_member_attribute)

In [67]:
# new mt_df, member and topic
mt_data = zip(new_member_id, new_member_attribute)

mt_df = pd.DataFrame(
    mt_data, columns = ['member_id', 'member_attribute'])

In [68]:
# re-caculate member_attribute_set
member_attribute_set = []
for i in range(member_amount):
  one_member_attribute = mt_df.iloc[i]['member_attribute']
  for j in range(len(one_member_attribute)):
    if one_member_attribute[j] not in member_attribute_set:
      member_attribute_set.append(one_member_attribute[j])
print(len(member_attribute_set))

1110


In [69]:
# matrix group-topic, GT
max_group_attribute_id = max(group_attribute_set)
GT= lil_matrix((max_group_id+1, max_group_attribute_id+1), dtype=np.int8)
for i in range(group_amount):
  GT_r = gt_df.iloc[i]['group_id']
  GT_c_set = gt_df.iloc[i]['group_attribute']
  for j in range(len(GT_c_set)):
    GT_c = GT_c_set[j]
    GT[GT_r, GT_c] = 1
GT.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int8)

In [70]:
# matrix member-topic, UT
max_member_attribute_id = max(member_attribute_set)
UT=lil_matrix((max_member_id+1, max_member_attribute_id+1), dtype=np.int8)
for i in range(member_amount):
  UT_r = mt_df.iloc[i]['member_id']
  UT_c_set = mt_df.iloc[i]['member_attribute']
  for j in range(len(UT_c_set)):
    UT_c = UT_c_set[j]
    UT[UT_r, UT_c] = 1
# UT.toarray()
# UT_df = pd.DataFrame(data=lil_matrix.todense(UT))
# UT_df.to_csv('UT.csv', index=False)

In [71]:
# matrix group_event, GE
GE = lil_matrix((max_group_id+1, max_event_id+1), dtype=np.int8)
for i in range(event_amount):
  GE_r = ge_df.iloc[i]['group_id']
  GE_c = ge_df.iloc[i]['event_id']
  GE[GE_r, GE_c] = 1
GE.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int8)

In [72]:
# TU_table, topic and member_set
member_set = [[] for i in range(len(member_attribute_set))]
for i in range(member_amount):
  TU_member_attribute_set = mt_df.iloc[i]['member_attribute']
  TU_member_id = mt_df.iloc[i]['member_id']
  for j in range(len(TU_member_attribute_set)):
    TU_member_attribute = TU_member_attribute_set[j]
    member_set_index = member_attribute_set.index(TU_member_attribute)
    member_set[member_set_index].append(TU_member_id)
   
tu_data = zip(member_attribute_set, member_set)

tu_df = pd.DataFrame(
    tu_data, columns = ['member_attribute', 'member_set'])
tu_df

,member_attribute,member_set
0,498,"[0, 35, 37, 62, 86, 158, 159, 184, 186, 189, 1..."
1,10485,"[0, 432, 475, 692, 908, 1251, 1393, 1400, 1800..."
2,2926,"[1, 2, 158, 163, 293, 689, 816, 835, 902, 926,..."
3,933,"[1, 4, 20, 26, 29, 111, 133, 152, 163, 169, 21..."
4,230,"[1, 4, 91, 96, 172, 260, 432, 433, 519, 770, 8..."
...,...,...
1105,27333,[44388]
1106,18409,[45111]
1107,22511,"[45941, 70762]"
1108,39519,[63026]


In [73]:
# matrix UTS
UTS_random_size = 10
UTS = lil_matrix((max_member_id+1, max_member_id+1), dtype=np.float16)
# for j in range(member_amount):
for x in range(UTS_random_size):
  j = random.choice(range(member_amount))
  j_attribute = mt_df.iloc[j]['member_attribute']
  j_member_id = mt_df.iloc[j]['member_id']
  for i in range(len(j_attribute)):
    member_set_index = member_attribute_set.index(j_attribute[i])
    k_member_set = member_set[member_set_index]
    for k in range(len(k_member_set)):
      k_member_id = k_member_set[k]
      if UTS[j_member_id, k_member_id] == 0:
        k_member_index = new_member_id.index(k_member_id)
        k_attribute = new_member_attribute[k_member_index]

        common_attribute = list(
            set(j_attribute).intersection(k_attribute))
        
        UTS_j_k = float(len(common_attribute)) / (
            len(j_attribute) + len(k_attribute) - len(common_attribute))
        
        UTS[j_member_id, k_member_id] = UTS_j_k
        UTS[k_member_id, j_member_id] = UTS_j_k
# UTS.toarray()

In [74]:
# member and member event set
UE_member_id = []
for i in range(len(train_set_1)):
  UE_member_id.append(mt_df.iloc[i]['member_id'])
UE_event_set = [[] for i in range(len(train_set_1))]
# for i in range(len(train_set_1)):
for i in range(1000):
  one_event_id = train_set_1.iloc[i]['event_id']
  UE_train_r_set = train_set_1.iloc[i]['yes_id']
  for j in range(len(UE_train_r_set)):
    one_member_id = UE_train_r_set[j]
    if one_member_id in UE_member_id: 
      member_index = UE_member_id.index(one_member_id)
      UE_event_set[member_index].append(one_event_id)

In [75]:
# matrix Sim
train_set_amount = len(train_set_1.index)
Sim_random_size = 5
x_Sim_random_size = Sim_random_size
Sim = lil_matrix((max_member_id+1, max_member_id+1), dtype=np.float16)
# for i in range(member_amount):
#   for j in range(member_amount):
for x in range(x_Sim_random_size):
  for y in range(Sim_random_size):
    i = random.choice(range(max_member_id+1))
    j = random.choice(range(max_member_id+1))
    sum_R_a = 0
    sum_R_b = 0
    for m in range(max_event_id+1):
      sum_R_a += UE_train[i, m]
      sum_R_b += UE_train[j, m]
    R_a = float(sum_R_a) / train_set_amount
    R_b = float(sum_R_b) / train_set_amount
    sim_numer = 0
    sim_denom_1 = 0
    sim_denom_2 = 0
    for n in range(max_event_id+1):
       sim_numer += ((UE_train[i, n] - R_a) * (UE_train[j, n] - R_b))
       sim_denom_1 += (UE_train[i, n] - R_a) ** 2
       sim_denom_2 += (UE_train[j, n] - R_b) ** 2
    sim_denom = math.sqrt(sim_denom_1) * math.sqrt(sim_denom_2)
    if sim_numer == 0 or sim_denom == 0: 
      behind_plus = 0
    else:
      behind_plus = float(sim_numer) / float(sim_denom)
    Sim[i, j] = UTS[i, j] + behind_plus
    Sim[j, i] = Sim[i, j]
  x_Sim_random_size -= 1
# Sim.toarray()

In [79]:
# Neighbor(a)
Neighbor_random_size = 3
index_member_id = []
for i in range(len(train_set_1)):
  index_member_id.append(mt_df.iloc[i]['member_id'])
K = 10
exist_member = len(index_member_id)
exist_k_set = [[] for i in range(exist_member)]
# for i in range(exist_member):
for i in range(Neighbor_random_size):
  i_id = index_member_id[i]
  index = index_member_id.index(i_id)
  Sim_row = Sim.getrow(i_id).toarray()[0].tolist()
  index_row = [*range(max_member_id+1)]
  unsorted_zip_list = list(zip(index_row, Sim_row))
  
  sorted_zip_list = sorted(
      unsorted_zip_list, key=lambda x: x[1], reverse=True)
  
  for j in range(K):
    exist_k_set[index].append(sorted_zip_list[j][0])